In [4]:
import ee
ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

# Define a function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 represent clouds and cirrus, respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    return image.updateMask(mask).divide(10000)

# Define the year of interest
year = '2023'

# Define the image collection and apply filters
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate(f'{year}-01-01', f'{year}-12-31')
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
           .map(maskS2clouds))

# Define visualization parameters
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# Load the region of interest from the specified asset
roi_medellin = ee.FeatureCollection("users/eeranyartrodrigo/datathonMedellin/roi_medellin").first().geometry()

# Define bands to select and the reducer for geometric median calculation
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
reducer = ee.Reducer.geometricMedian(6, 0.0001, 500)

# Compute the geomedian image
geomedian = dataset.select(bands).reduce(reducer).rename(bands)

# Export the geomedian image to an Earth Engine asset
export_task = ee.batch.Export.image.toAsset(
    image=geomedian.multiply(10000).toInt16(),
    assetId=f"users/eeranyartrodrigo/datathonMedellin/geomediana_medellin_{year}",
    maxPixels=1e12,
    region=roi_medellin,
    scale=10
)
export_task.start()
